In [1]:
# import torch_geometric
import sys
sys.path.insert(0, '../src')

In [2]:
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim as optim
import torchvision
import json
import argparse
# import cv2
import numpy as np
import torch
from torch.autograd import Function
from torchvision import models
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import time
from torchvision.datasets import CocoDetection
from PIL import Image
import pandas as pd
import sys
import pickle

In [3]:
sys.path.insert(0, '../src/')
from graph_network import GCNNet
from torch_geometric.data import DataLoader as TG_DataLoader

from dataset_class import *
from etl import *

sys.path.insert(0, '../src/data/cocoapi/PythonAPI')
from pycocotools.coco import COCO

In [4]:
# model_gcn=GCNNet(2048,80, [32]*2)

In [5]:
model_mlp=nn.Sequential(nn.Sequential(nn.Linear(2048,32),nn.ReLU(),nn.Dropout(p=0.3)),nn.Linear(32,80))

In [6]:
# model_gcn.cuda()

In [7]:
model_mlp

Sequential(
  (0): Sequential(
    (0): Linear(in_features=2048, out_features=32, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
  )
  (1): Linear(in_features=32, out_features=80, bias=True)
)

In [8]:
model_mlp.cuda()

Sequential(
  (0): Sequential(
    (0): Linear(in_features=2048, out_features=32, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
  )
  (1): Linear(in_features=32, out_features=80, bias=True)
)

In [9]:
# ! jupyter nbextension enable --py widgetsnbextension
# ! python -c "from torchvision import models; model = models.resnet50(pretrained=True)" 


In [10]:
pwd

'/home/jdlevy/COGS_185/cogs_185_final_project/notebooks'

In [11]:
os.getcwd()

'/home/jdlevy/COGS_185/cogs_185_final_project/notebooks'

In [12]:
# Loading in resnet pretrained
model = models.resnet50(pretrained=False)
state_dict=torch.load("../src/models/resnet50-19c8e357.pth")
model.load_state_dict(state_dict)
# model.fc = nn.Linear(2048, 80)
# Flatten to get 2048 embeddings
model.fc=nn.Flatten()

<All keys matched successfully>

In [14]:
# ! git clone https://github.com/cocodataset/cocoapi.git

In [15]:
import sys

In [16]:
from PIL import Image

In [17]:
# class cocoDataset(torch.utils.data.Dataset):
#     def __init__(self, root, annotation, transforms=None):
#         self.root = root
#         self.transforms = transforms
#         self.coco = COCO(annotation)

In [18]:
### TODO: add target transform to convert labels to proper labels using id_dict

In [19]:
# os.listdir('/datasets/COCO-2017/')

In [20]:
train_dir = '/datasets/COCO-2017/train2017'
val_dir = '/datasets/COCO-2017/val2017'

train_ann = '/datasets/COCO-2017/anno2017/instances_train2017.json'
val_ann = '/datasets/COCO-2017/anno2017/instances_val2017.json'

In [21]:
val_coco, val_data = load_data(val_ann)


loading annotations into memory...
Done (t=0.81s)
creating index...
index created!


In [22]:
pwd

'/home/jdlevy/COGS_185/cogs_185_final_project/notebooks'

In [23]:
with open('../src/data/id_dict.pkl', 'rb') as f:
    id_dict = pickle.load(f)

In [24]:
def get_transform():
    custom_transforms = []
    custom_transforms.append(torchvision.transforms.Resize(size=(128, 128)))
    custom_transforms.append(torchvision.transforms.ToTensor())
    return torchvision.transforms.Compose(custom_transforms)

In [25]:
train_dir = '/datasets/COCO-2017/train2017'
val_dir = '/datasets/COCO-2017/val2017'

train_ann = '/datasets/COCO-2017/anno2017/instances_train2017.json'
val_ann = '/datasets/COCO-2017/anno2017/instances_val2017.json'

clean_train_ann = '../data/temp/annotations/clean_instances_train2017.json'
clean_val_ann = '../data/temp/annotations/clean_instances_val2017.json'

In [26]:
def convert_ids(target):
    return id_dict[target]

In [27]:
%time train_set = cocoDataset(root=train_dir,\
                      annotation=clean_train_ann,\
                      transforms=get_transform(),\
                      target_transform=convert_ids)

%time val_set = cocoDataset(root=val_dir,\
                      annotation=clean_val_ann,\
                      transforms=get_transform(),\
                      target_transform=convert_ids)

loading annotations into memory...
Done (t=19.65s)
creating index...
index created!
CPU times: user 20.4 s, sys: 2.7 s, total: 23.1 s
Wall time: 23 s
loading annotations into memory...
Done (t=0.72s)
creating index...
index created!
CPU times: user 663 ms, sys: 102 ms, total: 765 ms
Wall time: 776 ms


In [28]:
## Params cell
num_epoch = 10
fname = 'resnet50_80_10'
train_batch_size = 12

In [35]:


trainloader = torch.utils.data.DataLoader(train_set,
                                          batch_size=train_batch_size,
                                          shuffle=True,
                                          num_workers=8,
                                          collate_fn=collate_fn)

valloader = torch.utils.data.DataLoader(val_set,
                                        batch_size=train_batch_size,
                                        shuffle=False,
                                        num_workers=8,
                                        collate_fn=collate_fn)



In [33]:
# for i, data in enumerate(trainloader, 0):
#     f = model(data[0])
#     print(f.shape)
#     o = model_mlp(f)
#     print(o.shape)
# #     print(data[0].shape)
# #     print(data[1].shape)
# #     print(data[2].shape)
#     print(len(data[3]))
#     break

In [36]:
# If there are GPUs, choose the first one for computing. Otherwise use CPU.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
# If 'cuda:0' is printed, it means GPU is available.

# model = models.resnet50(pretrained=False)
# state_dict=torch.load("../src/models/resnet50-19c8e357.pth")
# model.load_state_dict(state_dict)
# model.fc = nn.Linear(2048, 80)

model_mlp = nn.Sequential(nn.Sequential(nn.Linear(2048,32),nn.ReLU(),nn.Dropout(p=0.3)),nn.Linear(32,80))
model_mlp.to(device)
model.to(device)

print(f'model parameters sent to {device}')

loss_func = nn.CrossEntropyLoss(reduction='mean')
opt = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

avg_losses = [] # Average losses
epochs = num_epoch
print_freq = 100

for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # print(data)
        # Get the inputs.
        inputs, _, labels, _ = data
        
        
        
#       try:
        # Move the inputs to the specified device
        inputs, labels = inputs.to(device), labels.to(device)
        
        # First get 2048-D embeddings of our input through pre-trained resnet
        inputs = model(inputs)
        
        # Zero the parameter gradients.
        opt.zero_grad()

        # Forward step.
        outputs = model_mlp(inputs) # applying only our mlp to the model
        loss = loss_func(outputs, labels)

        # Backward step.
        loss.backward()

        # Optimization step (update the parameters).
        opt.step()

        # Print statistics.
        running_loss += loss.item()
#         except RuntimeError:
#             print(f'Input shape: {inputs.shape}\n Label shape: {labels.shape}')
#             print(f' Labels: {labels}')
#         print(i)
        if i % print_freq == print_freq - 1: # prints every several mini-batches.
            avg_loss = running_loss / print_freq
            print(f'[epoch: {epoch}, i: {i}] avg mini-batch loss: {avg_loss}')
            avg_losses.append(avg_loss)
            running_loss = 0.0
        
        del inputs, outputs, labels, loss
print('Finished Training.')

cuda:0
model parameters sent to cuda:0
[epoch: 0, i: 99] avg mini-batch loss: 4.361015701293946
[epoch: 0, i: 199] avg mini-batch loss: 4.353169994354248
[epoch: 0, i: 299] avg mini-batch loss: 4.343806920051574
[epoch: 0, i: 399] avg mini-batch loss: 4.329608383178711
[epoch: 0, i: 499] avg mini-batch loss: 4.325777707099914
[epoch: 0, i: 599] avg mini-batch loss: 4.307078580856324
[epoch: 0, i: 699] avg mini-batch loss: 4.285385375022888
[epoch: 0, i: 799] avg mini-batch loss: 4.254860320091248
[epoch: 0, i: 899] avg mini-batch loss: 4.221631302833557
[epoch: 0, i: 999] avg mini-batch loss: 4.165383124351502
[epoch: 0, i: 1099] avg mini-batch loss: 4.111093707084656
[epoch: 0, i: 1199] avg mini-batch loss: 4.064686479568482
[epoch: 0, i: 1299] avg mini-batch loss: 3.9829866695404053
[epoch: 0, i: 1399] avg mini-batch loss: 3.9377061319351196
[epoch: 0, i: 1499] avg mini-batch loss: 3.8634119391441346
[epoch: 0, i: 1599] avg mini-batch loss: 3.7918328714370726
[epoch: 0, i: 1699] avg 

[epoch: 1, i: 4199] avg mini-batch loss: 3.2147447633743287
[epoch: 1, i: 4299] avg mini-batch loss: 3.2598787546157837
[epoch: 1, i: 4399] avg mini-batch loss: 3.2415180516242983
[epoch: 1, i: 4499] avg mini-batch loss: 3.2261087536811828
[epoch: 1, i: 4599] avg mini-batch loss: 3.2079469847679136
[epoch: 1, i: 4699] avg mini-batch loss: 3.2217109036445617
[epoch: 1, i: 4799] avg mini-batch loss: 3.254996085166931
[epoch: 1, i: 4899] avg mini-batch loss: 3.2156294703483583
[epoch: 1, i: 4999] avg mini-batch loss: 3.192521963119507
[epoch: 1, i: 5099] avg mini-batch loss: 3.240382294654846
[epoch: 1, i: 5199] avg mini-batch loss: 3.177373456954956
[epoch: 1, i: 5299] avg mini-batch loss: 3.189113526344299
[epoch: 1, i: 5399] avg mini-batch loss: 3.2016156363487243
[epoch: 1, i: 5499] avg mini-batch loss: 3.27059951543808
[epoch: 1, i: 5599] avg mini-batch loss: 3.2239147663116454
[epoch: 1, i: 5699] avg mini-batch loss: 3.2098137855529787
[epoch: 1, i: 5799] avg mini-batch loss: 3.2058

[epoch: 2, i: 8299] avg mini-batch loss: 2.9890104079246522
[epoch: 2, i: 8399] avg mini-batch loss: 2.9475206542015076
[epoch: 2, i: 8499] avg mini-batch loss: 3.0265791058540343
[epoch: 2, i: 8599] avg mini-batch loss: 2.9526639366149903
[epoch: 2, i: 8699] avg mini-batch loss: 2.9165991139411926
[epoch: 2, i: 8799] avg mini-batch loss: 2.9834918189048767
[epoch: 2, i: 8899] avg mini-batch loss: 2.9235280394554137
[epoch: 2, i: 8999] avg mini-batch loss: 3.0011381912231445
[epoch: 2, i: 9099] avg mini-batch loss: 2.9727698945999146
[epoch: 2, i: 9199] avg mini-batch loss: 2.970596604347229
[epoch: 2, i: 9299] avg mini-batch loss: 3.005007631778717
[epoch: 2, i: 9399] avg mini-batch loss: 2.9386173725128173
[epoch: 2, i: 9499] avg mini-batch loss: 2.9876079249382017
[epoch: 2, i: 9599] avg mini-batch loss: 2.974640064239502
[epoch: 2, i: 9699] avg mini-batch loss: 2.9887929654121397
[epoch: 3, i: 99] avg mini-batch loss: 2.957038998603821
[epoch: 3, i: 199] avg mini-batch loss: 2.9514

[epoch: 4, i: 2699] avg mini-batch loss: 2.8163313007354738
[epoch: 4, i: 2799] avg mini-batch loss: 2.8485106348991396
[epoch: 4, i: 2899] avg mini-batch loss: 2.8095420885086058
[epoch: 4, i: 2999] avg mini-batch loss: 2.7328752446174622
[epoch: 4, i: 3099] avg mini-batch loss: 2.8032688522338867
[epoch: 4, i: 3199] avg mini-batch loss: 2.768956770896912
[epoch: 4, i: 3299] avg mini-batch loss: 2.8053554344177245
[epoch: 4, i: 3399] avg mini-batch loss: 2.7456996178627016
[epoch: 4, i: 3499] avg mini-batch loss: 2.802022864818573
[epoch: 4, i: 3599] avg mini-batch loss: 2.7758408999443054
[epoch: 4, i: 3699] avg mini-batch loss: 2.7902882623672487
[epoch: 4, i: 3799] avg mini-batch loss: 2.8144374227523805
[epoch: 4, i: 3899] avg mini-batch loss: 2.769060537815094
[epoch: 4, i: 3999] avg mini-batch loss: 2.790167076587677
[epoch: 4, i: 4099] avg mini-batch loss: 2.7225531220436094
[epoch: 4, i: 4199] avg mini-batch loss: 2.8038570046424867
[epoch: 4, i: 4299] avg mini-batch loss: 2.7

[epoch: 5, i: 6799] avg mini-batch loss: 2.631645300388336
[epoch: 5, i: 6899] avg mini-batch loss: 2.6557607007026673
[epoch: 5, i: 6999] avg mini-batch loss: 2.5920537972450255
[epoch: 5, i: 7099] avg mini-batch loss: 2.6421417927742006
[epoch: 5, i: 7199] avg mini-batch loss: 2.623697566986084
[epoch: 5, i: 7299] avg mini-batch loss: 2.6100093054771425
[epoch: 5, i: 7399] avg mini-batch loss: 2.6182085132598876
[epoch: 5, i: 7499] avg mini-batch loss: 2.6343050360679627
[epoch: 5, i: 7599] avg mini-batch loss: 2.676932631731033
[epoch: 5, i: 7699] avg mini-batch loss: 2.6258399581909178
[epoch: 5, i: 7799] avg mini-batch loss: 2.6444478023052214
[epoch: 5, i: 7899] avg mini-batch loss: 2.6077910017967225
[epoch: 5, i: 7999] avg mini-batch loss: 2.588411042690277
[epoch: 5, i: 8099] avg mini-batch loss: 2.5730313777923586
[epoch: 5, i: 8199] avg mini-batch loss: 2.7094161701202393
[epoch: 5, i: 8299] avg mini-batch loss: 2.657686324119568
[epoch: 5, i: 8399] avg mini-batch loss: 2.64

[epoch: 7, i: 1199] avg mini-batch loss: 2.4716108584403993
[epoch: 7, i: 1299] avg mini-batch loss: 2.4627025997638703
[epoch: 7, i: 1399] avg mini-batch loss: 2.503006423711777
[epoch: 7, i: 1499] avg mini-batch loss: 2.4925047826766966
[epoch: 7, i: 1599] avg mini-batch loss: 2.4988087260723115
[epoch: 7, i: 1699] avg mini-batch loss: 2.5065488505363462
[epoch: 7, i: 1799] avg mini-batch loss: 2.477833238840103
[epoch: 7, i: 1899] avg mini-batch loss: 2.5123038291931152
[epoch: 7, i: 1999] avg mini-batch loss: 2.425411809682846
[epoch: 7, i: 2099] avg mini-batch loss: 2.5334390687942503
[epoch: 7, i: 2199] avg mini-batch loss: 2.4546426737308504
[epoch: 7, i: 2299] avg mini-batch loss: 2.5214697277545928
[epoch: 7, i: 2399] avg mini-batch loss: 2.4562521505355837
[epoch: 7, i: 2499] avg mini-batch loss: 2.5007671892642973
[epoch: 7, i: 2599] avg mini-batch loss: 2.5042924344539643
[epoch: 7, i: 2699] avg mini-batch loss: 2.498035391569138
[epoch: 7, i: 2799] avg mini-batch loss: 2.5

[epoch: 8, i: 5299] avg mini-batch loss: 2.4319940638542175
[epoch: 8, i: 5399] avg mini-batch loss: 2.3878057289123533
[epoch: 8, i: 5499] avg mini-batch loss: 2.376921249628067
[epoch: 8, i: 5599] avg mini-batch loss: 2.3568664717674257
[epoch: 8, i: 5699] avg mini-batch loss: 2.3504626536369324
[epoch: 8, i: 5799] avg mini-batch loss: 2.3703112244606017
[epoch: 8, i: 5899] avg mini-batch loss: 2.377045508623123
[epoch: 8, i: 5999] avg mini-batch loss: 2.3608139789104463
[epoch: 8, i: 6099] avg mini-batch loss: 2.355445239543915
[epoch: 8, i: 6199] avg mini-batch loss: 2.381480348110199
[epoch: 8, i: 6299] avg mini-batch loss: 2.376917806863785
[epoch: 8, i: 6399] avg mini-batch loss: 2.294738233089447
[epoch: 8, i: 6499] avg mini-batch loss: 2.3485882449150086
[epoch: 8, i: 6599] avg mini-batch loss: 2.3172139954566955
[epoch: 8, i: 6699] avg mini-batch loss: 2.329444398880005
[epoch: 8, i: 6799] avg mini-batch loss: 2.3768014204502106
[epoch: 8, i: 6899] avg mini-batch loss: 2.3614

[epoch: 9, i: 9499] avg mini-batch loss: 2.213537555932999
[epoch: 9, i: 9599] avg mini-batch loss: 2.216568672657013
[epoch: 9, i: 9699] avg mini-batch loss: 2.2326023375988004
Finished Training.


In [ ]:
# ! ls /datasets/COCO-2017/val2017/ | grep 000000311913

In [37]:
# plot
len(avg_losses)

970

In [38]:
# '/datasets/COCO-2017/val2017'

In [39]:
fname


'resnet50_80_10'

In [41]:
# fname = '80_10_resnet50'


In [2]:
# model_mlp

In [ ]:
PATH = f'../src/models/{fname}.pth'
torch.save(model_mlp.state_dict(), PATH)
print(f'Model saved at {PATH}')

In [40]:

PATH = f'../src/models/{fname}.pth'
torch.save(model.state_dict(), PATH)
print(f'Model saved at {PATH}')

Model saved at ../src/models/resnet50_80_10.pth


In [44]:
### After model in cell above is saved, use model parameters w/ fc layer replaced to train GNN again and compare results...

In [41]:
pd.Series(avg_losses).to_csv(f'../data/out/avg_losses_{fname}.csv')

In [48]:
model.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [46]:
from torch_geometric.data import Data
from torch_cluster import knn_graph
graphs=[]
for i,(imgs,xy,y) in enumerate(valloader):
    print(imgs.shape,xy.shape,y.shape)
    edge_index = knn_graph(xy,k=5)
    graphs.append(Data(x=model(imgs),pos=xy,edge_index=edge_index,y=y))
#     print(graphs[i].x.shape)
    if i==10: break

torch.Size([1, 3, 128, 128]) torch.Size([1, 2]) torch.Size([1])


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same

In [83]:
from torch_geometric.data import DataLoader as TG_DataLoader
# https://pytorch.org/docs/stable/data.html#torch.utils.data.Subset

graph_dataloader=TG_DataLoader(graphs,batch_size=2,shuffle=True)#[0]

In [87]:
for G in graph_dataloader:
    y=G.y
    z=G.x
    edge_index=G.edge_index
    y_pred=model_gcn(z,edge_index)
    print(G)

NameError: name 'F' is not defined

In [89]:
# select device (whether GPU or CPU)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# DataLoader is iterable over Dataset
for imgs, annotations in valloader:
    imgs = list(img.to(device) for img in imgs)
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    print(annotations)

ValueError: too many values to unpack (expected 2)

In [17]:
# train_set

Dataset CocoDetection
    Number of datapoints: 118287
    Root location: /datasets/COCO-2017/train2017
    StandardTransform
Transform: <function crop_by_bbox at 0x7fe92a5d53b0>

In [20]:
val_set

Dataset CocoDetection
    Number of datapoints: 5000
    Root location: /datasets/COCO-2017/val2017
    StandardTransform
Transform: <function crop_by_bbox at 0x7fe92a5d53b0>

In [51]:
model_gcn=GCNNet(2048,80, [32]*2)

In [52]:
state_dict=torch.load("../src/models/graph_80_5_resnet50.pth")
model_gcn.load_state_dict(state_dict)
# model.fc = nn.Linear(2048, 80)

<All keys matched successfully>